In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 212.4MB 70kB/s 
     |████████████████████████████████| 204kB 20.0MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=4cf0d3a044dd82c7703a406ca3fb194640e290d8af74a15503164a04f347154c
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 36.5 MB of archives.
After this 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
id='1f_9EbnywCj35EBUA32sueigxBjBJwALr'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('War and Peace by Leo Tolstoy (ru).txt')

In [ ]:
import pandas as pd

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().set("spark.ui.port", "4050")

sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
spark

*`ngrok может отработать не с первого раза, повторите при необходимости.`*

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-05-31 09:02:06--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 35.170.10.11, 52.200.57.5, 54.164.22.162, ...
Connecting to bin.equinox.io (bin.equinox.io)|35.170.10.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  15.1MB/s    in 0.9s    

2021-05-31 09:02:07 (15.1 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://e84e7b621e0b.ngrok.io


###### Подсчитай кол-во слов в документе "War and Peace by Leo Tolstoy (ru).txt"

In [ ]:
df = spark.read.csv('./War and Peace by Leo Tolstoy (ru).txt',
                    encoding='cp1251', sep='\n')

In [ ]:
import re
import unicodedata

def words(row):
  string = unicodedata.normalize('NFKD', row[0])
  return re.sub(r'[^\w\s]', '', string).split()

In [ ]:
df.rdd.flatMap(words).count()

472901

###### Необходимо обучить модель используя Spark MLlib (модель на ваш выбор, например Decision Tree) и получить accuracy.
Подробнее тут: https://spark.apache.org/docs/latest/ml-classification-regression.html

In [ ]:
id='13yfAoONwq4rS5XrTv3IrcqcFcdgfvK9V'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-train.txt')

id='1VE_9x0LQvOJpHXbXp_RMPl3Q4wRUuOok'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('mnist-digits-test.txt')

In [ ]:
training = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-train.txt")
test = spark.read.format("libsvm") \
  .option("numFeatures", "784") \
  .load("./mnist-digits-test.txt")

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")
model = dt.fit(training)

In [ ]:
predictions = model.transform(test)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(accuracy)

0.6792


###### Тут лежат данные для решения задач на sql. Необходимо написать код который формирует из данных датафреймы и используя sqlContext написать sql код с решением. Приветствуется наличие нескольких вариантов решения задачи.

In [ ]:
id='1kUIrskM0zNH8u71G9M1BkHjRQYxvgAvh'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('data.zip')
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: data/regions.csv        
  inflating: data/departments.csv    
  inflating: data/jobs.csv           
  inflating: data/locations.csv      
  inflating: data/country.csv        
  inflating: data/employees.csv      
  inflating: data/job_history.csv    


In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
df_regions = sqlContext.read.csv('data/regions.csv', header=True, inferSchema=True, sep='\t', quote='"')
df_regions.registerTempTable('regions')

df_departments = sqlContext.read.csv('data/departments.csv', header=True, inferSchema=True, sep='\t', quote='"')
df_departments.registerTempTable('departments')

df_jobs = sqlContext.read.csv('data/jobs.csv', header=True, inferSchema=True, sep='\t', quote='"')
df_jobs.registerTempTable('jobs')

df_locations = sqlContext.read.csv('data/locations.csv', header=True, inferSchema=True, sep='\t', quote='"')
df_locations.registerTempTable('locations')

df_country = sqlContext.read.csv('data/country.csv', header=True, inferSchema=True, sep='\t', quote='"')
df_country.registerTempTable('country')

df_employees = sqlContext.read.csv('data/employees.csv', header=True, inferSchema=True, timestampFormat='dd.MM.yy', sep='\t', quote='"') \
  .withColumn("HIRE_DATE", date_format('HIRE_DATE', "yyyy-MM-dd").cast('date'))
df_employees.registerTempTable('employees')

df_job_history = sqlContext.read.csv('data/job_history.csv', header=True, inferSchema=True, timestampFormat='dd.MM.yy', sep='\t', quote='"') \
  .withColumn("START_DATE", date_format('START_DATE', "yyyy-MM-dd").cast('date')) \
  .withColumn("END_DATE", date_format('END_DATE', "yyyy-MM-dd").cast('date'))
df_job_history = df_job_history.withColumn('START_DATE',
                                           when(year('START_DATE') > 2050,
                                                add_months(df_job_history['START_DATE'], -1200)) \
                                           .otherwise(df_job_history['START_DATE']))
df_job_history.registerTempTable('job_history')

Кто получает больше всего? Кто меньше всего?

In [ ]:
sqlContext.sql('''
SELECT Employee_id, First_name, Last_name, Salary
FROM employees
WHERE salary = (SELECT MAX(salary) FROM employees)
''').show()

sqlContext.sql('''
SELECT Employee_id, First_name, Last_name, Salary
FROM employees
WHERE salary = (SELECT MIN(salary) FROM employees)
''').show()

+-----------+----------+---------+------+
|Employee_id|First_name|Last_name|Salary|
+-----------+----------+---------+------+
|        100|    Steven|     King| 24000|
+-----------+----------+---------+------+

+-----------+----------+---------+------+
|Employee_id|First_name|Last_name|Salary|
+-----------+----------+---------+------+
|        132|        TJ|    Olson|  2100|
+-----------+----------+---------+------+



Выведете топ 5 по зарплате.

In [ ]:
sqlContext.sql('''
SELECT * FROM (
  SELECT
    ROW_NUMBER() OVER (ORDER BY salary DESC) as Num,
    Employee_id,
    first_name ||' '|| last_name as  Name,
    Salary
  FROM employees)
WHERE num <= 5 
''').show()

+---+-----------+--------------+------+
|Num|Employee_id|          Name|Salary|
+---+-----------+--------------+------+
|  1|        100|   Steven King| 24000|
|  2|        101| Neena Kochhar| 17000|
|  3|        102|   Lex De Haan| 17000|
|  4|        145|  John Russell| 14000|
|  5|        146|Karen Partners| 13500|
+---+-----------+--------------+------+



In [ ]:
sqlContext.sql('''
SELECT
  Employee_id,
  first_name ||' '|| last_name as  Name,
  Salary
FROM employees
ORDER BY Salary DESC
LIMIT 5
''').show()

+-----------+--------------+------+
|Employee_id|          Name|Salary|
+-----------+--------------+------+
|        100|   Steven King| 24000|
|        101| Neena Kochhar| 17000|
|        102|   Lex De Haan| 17000|
|        145|  John Russell| 14000|
|        146|Karen Partners| 13500|
+-----------+--------------+------+



Сколько всего регионов? Сколько работников в каждом регионе?

In [ ]:
sqlContext.sql('''
SELECT COUNT (*) as num_of_regions
FROM regions
''').show()

+--------------+
|num_of_regions|
+--------------+
|             4|
+--------------+



In [ ]:
sqlContext.sql('''
SELECT COUNT(DISTINCT region_id) as regions_with_employees
FROM employees e
JOIN departments d
  ON e.department_id = d.department_id
JOIN locations l
  ON d.location_id = l.location_id
JOIN country c
  ON l.country_id = c.country_id
''').show()

+----------------------+
|regions_with_employees|
+----------------------+
|                     2|
+----------------------+



In [ ]:
sqlContext.sql('''
SELECT region_name, COUNT(*) as num_of_employees
FROM employees e
JOIN departments d
  ON e.department_id = d.department_id
JOIN locations l
  ON d.location_id = l.location_id
JOIN country c
  ON l.country_id = c.country_id
JOIN regions r
  ON c.region_id = r.region_id
GROUP BY region_name
''').show()

+-----------+----------------+
|region_name|num_of_employees|
+-----------+----------------+
|     Europe|              36|
|   Americas|              70|
+-----------+----------------+



Выведете всех работников из Китая.

In [ ]:
sqlContext.sql('''
SELECT employee_id, first_name ||' '|| last_name as  Name
FROM employees e
JOIN departments d
  ON e.department_id = d.department_id
JOIN locations l
  ON d.location_id = l.location_id
JOIN country c
  ON l.country_id = c.country_id
WHERE country_name = 'China'
''').show()

+-----------+----+
|employee_id|Name|
+-----------+----+
+-----------+----+



Укажите самую высокооплачиваемою должность.

In [ ]:
# Среди всех должностей
sqlContext.sql('''
SELECT job_title, min_salary, max_salary
FROM jobs
WHERE min_salary = (SELECT MAX(min_salary) FROM jobs)
''').show()

+---------+----------+----------+
|job_title|min_salary|max_salary|
+---------+----------+----------+
|President|     20080|     40000|
+---------+----------+----------+



In [ ]:
# Среди представленных работников
sqlContext.sql('''
SELECT job_title, salary
FROM employees e
JOIN jobs j
  ON e.job_id = j.job_id
WHERE salary = (SELECT MAX(salary) FROM employees)
''').show()

+---------+------+
|job_title|salary|
+---------+------+
|President| 24000|
+---------+------+



Выведете всех работников связанных с ИТ. Выведете их менеджеров. 

In [ ]:
sqlContext.sql('''
SELECT
  e1.employee_id,
  e1.first_name ||' '|| e1.last_name as employee_name,
  e2.first_name ||' '|| e2.last_name as manager_name
FROM employees e1
JOIN employees e2
  ON e1.manager_id = e2.employee_id
WHERE e1.JOB_ID = 'IT_PROG'
''').show()

+-----------+----------------+----------------+
|employee_id|   employee_name|    manager_name|
+-----------+----------------+----------------+
|        103|Alexander Hunold|     Lex De Haan|
|        104|     Bruce Ernst|Alexander Hunold|
|        105|    David Austin|Alexander Hunold|
|        106| Valli Pataballa|Alexander Hunold|
|        107|   Diana Lorentz|Alexander Hunold|
+-----------+----------------+----------------+



Выведете имя и фамилию работника, его текущую и предыдущую должности и сколько полных недель и дней прошло с момент изменения. 



In [ ]:
sqlContext.sql('''
SELECT
  e.employee_id,
  start_date, end_date, hire_date,
  j1.job_title as current_job,
  j2.job_title as previous_job
FROM (
  SELECT
    employee_id, start_date, end_date, job_id,
    ROW_NUMBER() OVER (PARTITION BY employee_id ORDER BY end_date DESC) as rn
  FROM job_history) jh 
JOIN employees e
  ON jh.employee_id = e.employee_id
JOIN jobs j1
  ON e.job_id = j1.job_id
JOIN jobs j2
  ON jh.job_id = j2.job_id
WHERE rn=1
''').show()

+-----------+----------+----------+----------+--------------------+--------------------+
|employee_id|start_date|  end_date| hire_date|         current_job|        previous_job|
+-----------+----------+----------+----------+--------------------+--------------------+
|        101|2001-10-28|2005-03-15|2005-09-21|Administration Vi...|  Accounting Manager|
|        102|2001-01-13|2006-07-24|2001-01-13|Administration Vi...|          Programmer|
|        114|2006-03-24|2007-12-31|2002-12-07|  Purchasing Manager|         Stock Clerk|
|        122|2007-01-01|2007-12-31|2003-05-01|       Stock Manager|         Stock Clerk|
|        176|2007-01-01|2007-12-31|2006-03-24|Sales Representative|       Sales Manager|
|        200|2002-07-01|2006-12-31|2003-09-17|Administration As...|   Public Accountant|
|        201|2004-02-17|2007-12-19|2004-02-17|   Marketing Manager|Marketing Represe...|
+-----------+----------+----------+----------+--------------------+--------------------+



Выведете уникальные телефонные номера

In [ ]:
sqlContext.sql('''
SELECT DISTINCT phone_number
FROM employees
ORDER BY phone_number
''').show()

+------------------+
|      phone_number|
+------------------+
|011.44.1343.329268|
|011.44.1343.529268|
|011.44.1343.629268|
|011.44.1343.729268|
|011.44.1343.829268|
|011.44.1343.929268|
|011.44.1344.129268|
|011.44.1344.345268|
|011.44.1344.429018|
|011.44.1344.429268|
|011.44.1344.429278|
|011.44.1344.467268|
|011.44.1344.478968|
|011.44.1344.486508|
|011.44.1344.498718|
|011.44.1344.619268|
|011.44.1344.987668|
|011.44.1345.429268|
|011.44.1345.529268|
|011.44.1345.629268|
+------------------+
only showing top 20 rows



Есть ли сотрудники с одинаковыми фамилиями и сколько их.

In [ ]:
sqlContext.sql('''
SELECT * FROM (
  SELECT last_name,
  count(*) as num,
  CONCAT_WS(', ', COLLECT_LIST(first_name || ' ' || last_name)) as full_names
  FROM employees
  GROUP BY last_name)
WHERE num > 1
''').show(truncate=False)

+---------+---+-----------------------------------+
|last_name|num|full_names                         |
+---------+---+-----------------------------------+
|Smith    |2  |Lindsey Smith, William Smith       |
|King     |2  |Steven King, Janette King          |
|Cambrault|2  |Gerald Cambrault, Nanette Cambrault|
|Taylor   |2  |Jonathon Taylor, Winston Taylor    |
+---------+---+-----------------------------------+

